This notebook compares annotations using different GLLMs with codebooks based prompts of Boukes 2024, Jaidka 2022 and Naab 2025 on their respective datasets

In [1]:
import requests
import os
from dotenv import load_dotenv
load_dotenv("sjoerdAzure.env")  # Load environment variables from .env file
import time

import typing

from sklearn.metrics import cohen_kappa_score, classification_report
import krippendorff
import yaml

import pandas as pd

import config
import src
import tqdm
import json
import numpy as np
import logging

#import cltrier_lib as lib
import pyreadstat
import yaml
pd.set_option('display.max_colwidth', 100) 
#set up helper variables and functions:
CFG = config.Config()


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\sstolwi\Github\llmdiv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\sstolwi\Github\llmdiv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\sstolwi\Github\llmdiv\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.start()
  File "c:\Users\sstolwi\Git

In [7]:
#load data:

# Naab2025 data
naab = pd.read_parquet(f'{CFG.report_dir}/Naab2025.parquet')
# Jaida2024 data
jaidka = pd.read_parquet('data/jaidka2022/TwitterDeliberativePolitics2.parquet')
# Boukes
boukes = pd.read_parquet('data/publicsphere/publicsphere.cardiff_prompt_classify_anon.parquet')
boukesT = pd.read_csv('data/publicsphere/full_data.csv')
#MH_clemm 2024
MHclemm = pd.read_parquet('data/MH_BClemm_data/germany_val_all_llama.parquet') #uses same prompt as Boukes, but different data (and here we use German, but could ask for English and (?) Spanish (?))

list the variables we want to use:
**rationality** - prompt: 'rationality_simple2', 'rationality_jaidka',        
  manual coding: "Justification" (Jaidka), RATIONALITY_DUMMY
**incivility** - prompt: 'incivility_simple2', 'incivility_jaidka',  civility_jaidka         
  manual coding: INCIVILITY_DUMMY, Incivility_tot ('Uncivil_abuse', 'Empathy_Respect'), Uncivil_abuse, "Empathy_Respect" (jaidka)
**interactivity** - prompt: 'interactivity_acknowledgement_simple2', interactivity_acknowledgement_jaidka       
  manual coding: INTERACTIVITY_DUMMY, Reciprocity (Jaidka)
**diversity/ideology** - prompt: 'political_ideology_US', 'political_ideology' (german)  -> no ideology in Jaidka
  manual coding: LIBERAL_DUMMY, CONSERVATIVE_DUMMY
**political_dum** - prompt: 'political_post', political_post_jaidka 
  manual coding: HAS_OPINION_DUMMY

In [ ]:
#model variants:
# llama31_8b
# llama31_70b
# gpt4o
# gpt4Turbo

#optional:
# gpt4 (OPenAI, microsoft)
# llama33_70b (Meta)
# Gemma3:22b (US, google) (based on Gemini 2)
# "id":"deepseek-r1:70b","name":"DeepSeek-R1 (china)
# qwen2.5:70b (china)
# mistral-large:123b","name":"Mistral" (europe)


list the annotations we have available per dataset:
**Jaidka**: 'rationality_simple2_Llama8b_dum', 'rationality_simple2_gpt4o_dum','rationality_jaidka_Llama8b_dum','rationality_jaidka_gpt4o_dum', 'civility_jaidka_gpt4o_dum', incivility_simple2_gpt4o_dum, incivility_jaidka_gpt4o_dum, reciprocity_jaidka_gpt4o_dum, interactivity_acknowledgement_simple_gpt4o_dum, political_post_jaidka_gpt4o_dum, political_post_gpt4o_dum
**Boukes**: 
*rationality*: rationality_simple2_dum (+ rationality_simple_dum, rationality_combine_dum, rationality_combine_exactexample_dum, rationality_prompt_dum (aggregation of indicator prompt scores)), rationality_simple2_gpt4o_system_dum, rationality_simple2_gpt4T_system_dum, rationality_simple2_small_dum, 
*incivility*: incivility_simple2_dum (+ incivility_simple_dum, incivility_combine_dum), incivility_prompt_dum (aggregation of indicator prompt scores), incivility_simple2_gpt4o_system_dum, incivility_simple2_gpt4T_system_dum, incivility_simple2_small_dum 
*interactivity*: interactivity_acknowledgement_simple_dum (+ interactivity_acknowledgement_simple2_dum), interactivity_acknowledgement_simple_gpt4o_system_dum, interactivity_acknowledgement_simple_gpt4T_system_dum, interactivity_acknowledgement_simple_small_dum (+interactivity_acknowledgement_simple_small2_dum)
*diversity*: political_liberal_US_dum, political_conservative_US_dum, political_liberal_US_gpt4o_system_dum, political_liberal_US_gpt4T_system_dum, political_conservative_US_gpt4o_system_dum, political_conservative_US_gpt4T_system_dum, political_liberal_US_small_dum, political_conservative_US_small_dum
*political_dum*: political_opinion_US_dum, political_opinion_US_gpt4o_system_dum, political_opinion_US_gpt4T_system_dum, political_opinion_US_small_dum (either liberal/conservative; Boukes)

#to be made prompts:
#Naab:
 Argumente/ v75 - rationality in combination with quellenbelege?
(Operationalisierung erstellt in Anlehnung an Ziegele & Quiring, 2015)
Argumente sind Aussagen, die dazu dienen sollen, Behauptungen zu begründen oder zu widerlegen. Es wird kodiert, ob ein Nutzerkommentar Argumente verwendet, um eine oder mehrere geäußerte(n) Meinung(en) zu begründen.
Von Interesse ist hier, ob die Nutzer ihre eigenen Aussagen mit Argumenten unterstützen oder ob diese unbegründet bleiben. Der Leser muss sich bei zusammenhängenden Aussagen im Kommentar fragen, ob die Frage nach dem „warum“ beantwortet wird. Anders gesagt: Gut erkennbar ist ein Argument, wenn man es problemlos mit einem Kausalsatz (weil...) an eine Behauptung anfügen kann oder wenn man eine begründende „Wenn-Dann-Beziehung“ zwischen den Sätzen herstellen kann. Eine Aneinanderreihung von Aussagen zählt nicht als Argumentation.
! Achtung: Es geht hier nicht um die Qualität der Argumente! Es wird nur kodiert, ob Begründungen für Aussagen/Behauptungen/Positionen angeführt werden (s. Beispiele)!
0
Keine Argumente
Im Kommentar werden keine Argumente verwendet. Behauptungen werden nicht begründet.
Der Abtreibungsparagraph muss abgeschafft werden.
Alles ist wichtiger als Fußball.
Schade, dass die FDPs in Hessen nochmal geschafft hat aber für Schwarz-Gelb reicht‘s trotzdem nicht mehr. Dafür ist neben rot-rot-grün jetzt auch ‘ne klassische Ampel möglich. (Fragen nach dem „Warum“ bleiben unbeantwortet, Sätze lassen sich nicht mit „weil“ verknüpfen)
1
Mind. ein Argument
Im Kommentar wird mindestens ein Argument, u.U. auch mehrere Argumente, formuliert.
Deutschland hätte Snowden schon lange Asyl anbieten müssen: Schließlich ist er ein politisch Verfolgter, dessen Gesundheit auf dem Spiel steht, wenn er von seinen wildgewordenen Landsleuten aufgegriffen werden sollte.
In erster Linie pflichte ich der GDL bei, (denn) sie demonstrieren wie es geht. In zweiter Linie möchte ich den Arbeitgebern das Armutszeugnis ausstellen, (denn) ich hab zum ersten Mal das Gefühl dass einer das richtige macht.
62
99
Nicht eindeutig zuzuordnen.


Quellenbelege im Kommentar/ v76
Es wird kodiert, ob im Kommentar nachprüfbare Belege für Fakten, Tatsachenbehauptungen, Argumente oder Meinungen genannt werden. Dies passiert, indem auf Quellen verwiesen wird. Ein Argument, eine Prognose oder eine Werthaltung alleine sind kein Beleg. Der Beleg macht eine Aussage prinzipiell für jeden intersubjektiv nachprüfbar. Ein Verweis auf Quellen ist ausreichend, die Quellen müssen nicht (können aber) verlinkt sein. Werden zwar Fakten genannt, diese sind jedoch nicht belegt und nur mit größerem Aufwand intersubjektiv prüfbar, zählt dies nicht als Beleg.
0
Kein Beleg genannt
Ich glaube, dass auch der Asylmissbrauch ein Problem ist. Aber sowas ist auch schwer zu erkennen.
Assad wird weiter Chemiewaffen einsetzen, er weiß, er kann sich auf seine russischen und chinesischen Freunde verlassen. (Spekulation und Zukunftsprognosen sind keine Fakten)
Alle außer Syrien und Iran haben die Amis im Sack und wenn das Problemchen gelöst wurde, dann wird wohl Russland spüren wer Herr im Hause ist und das ist das eigentliche Ziel der USA.
In Frankreich gilt ein Mindestlohn von 11,50€, auch das trägt dazu bei, dass Deutschland mit seinem Niedriglohn den europäischen Nachbarn schwer zusetzt!
Der Fingerabdrucksensor hat nichts mit NSA zu tun. Wer das denkt ist auf dem Holzweg. Der Fingerabdrucksensor ist ja nur für die Entsperrung des Telefons. 64Bit ist nice to have.
1
Mindestens ein Quellen-beleg genannt
Der Kommentator enthält Verweise auf Quellen, mit
Die Tagesschau meldete gestern neben den 20 Toten auch über 70 Verletzte.
63
denen Fakten oder sonstige Aussagen belegt werden. Dadurch sind sie prinzipiell für jeden intersubjektiv nachprüfbar. Ein Verweis auf Quellen ist ausreichend, sie müssen nicht (können aber) verlinkt sein.
Die CDU kommt auf 41,5 Prozent, SPD, Grüne und Linke zusammen auf 42, 7. Und das kann sich jedes Kind auf Wikipedia ansehen: https://de.wikipedia.org/wiki/Bundestagswahl_2013
99
Nicht eindeutig zuzuordnen.


Inziviles im Kommentar/ v711 - check definition
(Operationalisierung erstellt in Anlehnung an Ziegele & Quiring, 2015; vgl. auch Coe, Kenski & Rains, 2014; Papacharissi, 2004)
Inzivilität wird kodiert, wenn im Kommentar die folgenden Aussagetypen vorkommen, die einen unnötig respektlosen Ton gegenüber den Diskussionsteilnehmern, Themen, nicht-anwesenden Dritten oder Journalisten/Medien transportieren:
•
Beschimpfungen: Bösartige oder herabsetzende Wörter, die an Personen oder Personengruppen gerichtet sind.
o
Das verachtenswerte politische Unkraut FDP ist jetzt dort, wo es hingehört. (Thema Landtagswahl in Hessen)
•
Abfällige Bewertungen: Bösartige oder herabsetzende Wörter und Sätze, die gegen eine Idee, einen Plan, ein Verhalten oder eine Politik gerichtet sind.
70
o
Klassisches Eigentor geschossen mit diesem Schrott Handy! (Thema iPhone-Vorstellung)
•
Lügen: Explizite oder implizite Aussagen, dass eine Idee, ein Plan, ein Verhalten oder eine Politik unehrlich sind.
o
Und welche "Glaubwürdigkeit" hat ein Präsident, der die Dauer-Lügen seiner Geheimdienste vertritt, ÜBERHAUPT noch? (Thema Syrien-Konflikt)
•
Vulgarität: Obszönitäten oder eine vulgäre Sprache, die im persönlichen Gespräch unangebracht wäre.
o
Es gibt genug Spinner, die ihre eigene unzureichende Schwanzlänge kompensieren müssen. (Thema iPhone-Vorstellung)
•
Abschätzige Bemerkungen: Herabsetzende Bemerkungen über die Art und Weise, wie eine Person auftritt oder kommuniziert.
o
Damit fühlst du dich nun überlegen und toll, hm? Infantiles Gehabe und Profilierung. (Thema Veggie-Day)
•
Stereotype: Eine vereinfachende, verallgemeinernde, schematische Reduzierung einer Erfahrung, Meinung oder Vorstellung auf ein meist verfestigtes, oft ungerechtfertigtes und emotional aufgeladenes Vorurteil. Stereotype bilden eine Voraussetzung für die Diskriminierung von Minderheiten und die Ausbildung von Feindbildern, Rassismus und Sexismus. Oftmals – aber nicht immer – ist die Kommunikation von Stereotypen politisch unkorrekt. So werden zum Beispiel einzelne Personen mit einer Gruppe gleichgesetzt, oder auf die Haupteigenschaften einer Gruppe reduziert.
o
Der Deutsche hasst den Mainstream, kauft bei KIK und Aldi, schmeißt 2 Jahre alte Handys auf den Müll und fährt mit Monatskarte zur Atomkraftdemo. So ambivalent spießig wie der/die Deutsche ist wohl keine Nation.
o
Frauen sind zum Putzen da!
o
Du bist doch ein Hauptschüler, so beschränkt wie du dich ausdrückst.
•
Absprechen von Rechten (individuell): Anderen Diskussionsteilnehmern werden Menschen- und Persönlichkeitsrechte abgesprochen
z. B. Angriffe auf die Meinungsfreiheit oder persönliche Freiheit (Recht, Leben frei zu gestalten, auf finanziellen Wohlstand und gesicherte Existenz), Androhung von Gewalt, einschließlich impliziter Drohungen.
o
Dir gehört der Mund verboten!; Leute mit Einstellungen wie deiner sollten nicht wählen dürfen.
o
Die sollten alle im Meer ersaufen;
•
Bedrohung der Demokratie/ Gefährdung demokratischer Werte (gesellschaftlich): Der Kommentar trifft Aussagen, die vermuten lassen, dass die demokratischen Werte/ die Demokratie nicht wertgeschätzt oder diese gar bedroht werden.
! Achtung: Die Artikulation von Kritik und Meinungsverschiedenheiten wird nicht als Inzivilität kodiert, sofern diese Artikulation angemessen höflich und sachlich erfolgt. In anderen Worten: Inzivilität ist nicht die Artikulation von Kritik und Meinungsverschiedenheiten an sich, sondern eine unangemessene Art und Weise, diese zu formulieren.
0
Keine Inzivilität
Der Kommentar ist zivil und höflich formuliert. Er transportiert keinen unnötig respektlosen Ton ggü. Anderen bzw. enthält keine der oben genannten Aussagetypen.
71
1
Vereinzelt inzivil
Der Kommentar ist überwiegend zivil und höflich formuliert. Er enthält jedoch vereinzelt die o.g. Aussagetypen.
2
Überwiegend / ausschließlich inzivil
Der Kommentar ist kaum oder nicht zivil und höflich formuliert. Er enthält überwiegend oder ausschließlich die o.g. Aussagetypen.
99
Nicht eindeutig zuzuordnen.


Fragen/ v7135 -> interactivity if either?
(Operationalisierung erstellt in Anlehnung an Ziegele & Quiring, 2015)
Hier wird kodiert, ob Autor innerhalb eines Kommentars Informations-, Wissens- oder Verständnisdefizite aktiv kommunizieren und ausgleichen wollen, oder ob rhetorische Fragen gestellt werden, auf die eigentlich keine Antwort erwünscht ist. Es wird kodiert, ob überhaupt Fragen gestellt werden, nicht aber, an wen diese ggf. gerichtet sind.
a)
Echte Fragen
Als echte Fragen werden alle Aussagen betrachtet, die mit einem Fragezeichen enden oder von ihrer Satzstruktur her als Frage aufgefasst werden können und die ein echtes Informationsbedürfnis anzeigen. Nur wenn eindeutig erkennbar ist, dass der Autor am Erhalt von Informationen/Meinungen interessiert ist, wird die Frage als „echt“ kodiert. Typen von „echten“ Fragen sind u. a. Wissens-, Einstellungs- und Verständnisfragen.
Bsp.: Kennt sich hier einer mit dem Völkerrecht aus? Wieso kann Obama überhaupt überlegen einen ‘Alleingang‘ zu starten?
b)
Rhetorische Fragen
Enthält der Kommentar eine oder mehrere rhetorische Fragen? Rhetorische Fragen werden vor allem als Argumentationsstrategie einsetzt, um die eigenen Positionen durchzusetzen. Sie zielen nicht auf Informationsgewinn ab, sondern der Autor will vorrangig Reaktionen wie Zustimmung bzw. Ablehnung erhalten.
! Achtung: Rhetorische Fragen zusätzlich beim Diskussionsfaktor Kontroverse kodieren.
Kodieranweisung: Rhetorische Frage schlägt echte Frage → sobald eine rhetorische Frage formuliert wird, wird eine „2“ kodiert.

maybe do the following analyses:
-baseline differences between slightly differing prompts on Boukes - best vs long vs best with slight word change vs long exact example vs prompt_dum (indicator aggregation) -> only have this for Llama3.1:70b
-model differences on Boukes - best prompt Llama3.1:70b vs 8b vs GPT4T vs GPT4o
-codebook differences on Boukes - Jaidka vs Boukes 
-dataset differences - best prompt Llama3.1:70b vs 8b vs GPT4o Jaidka vs Boukes -> need to run Llama3.1:70b on Jaidka, but no longer available via Trier -> could run both datasets with Llama3.3:70b instead
-error analysis - how do differences between models overlap with differences between models and manual data
-downstream effect on Boukes2024

maybe leave out indicator aggregation prompt, only focus on best vs slight word change vs arbitrary change (Barrie ea 2025 prompt stability),  try run baseline for Boukes and Jaidka with GPT4o (azure) and Llama3.3:70b (Trier)
compare model differences on Boukes also for other models if possible (but at least add L3.3:70b)

drop Naab data? 
ask MH for english data for out of sample comparison on political_dum and ideology
note we only have low-temperature anotations for Llama available

#TO DO:
-classify Boukes, Jaidka and MHdata with options_low, Llama3.3:70b for Boukes and Jaidka prompts
-classify Boukes with prompt stability/slightly different prompts with Llama3.3:70b and gpt4o -> use Barrie ea 2025 approach of paraphrasing prompt with increasing temperature, perhaps use Llama instead of pegasus for ease of use? - report prompt variations in appendix for manual validation
-error analysis
-downstream effects on Boukes

In [32]:
#set up helper variables and functions:
CFG = config.Config()

def load_json(path: str):
    with open(path, encoding='utf-8') as fp:
        return json.load(fp)
    
#set option variables:

#set options to low temperature (0,1):
options_low_str = """
seed: 42
temperature: 0.1
"""

options_low = yaml.safe_load(options_low_str)

#apparently the 3.1 70b model is no longer available via Trier...
MODELsmall: str = 'llama3.1:8b'
MODEL33large: str = 'llama3.3:70b' # options: 'gemma:7b-instruct-q6_K', 'gemma2:27b-instruct-q6_K', 'llama3.1:8b-instruct-q6_K', 'llama3.1:70b-instruct-q6_K', 'mistral:7b-instruct-v0.3-q6_K', 'mistral-large:123b-instruct-2407-q6_K', 'mixtral:8x7b-instruct-v0.1-q6_K', 'mixtral:8x22b-instruct-v0.1-q6_K', 'phi3:14b-medium-128k-instruct-q6_K' or 'qwen2:72b-instruct-q6_K'
MODELgpt4o = "nf-gpt-4o-2024-08-06" # in principe is er nu van elk model een nf (no filter) en een normale versie beschikbaar, de no filter versies zijn alleen voor onderzoekers beschikbaar voor analyze van content die niet door de filter heen zou komen.
MODELgpt4T = "nf-gpt-4-turbo" # Can be gpt-35-turbo, gpt-4-turbo, gpt-4 or Meta-Llama-3-8B-Instruct.
MODEL33largeAzure = 'Llama-3.3-70B-Instruct' #azureml://registries/azureml-meta/models/Llama-3.3-70B-Instruct/versions/4 / options:  "data": [
MODEL31largeAzureNF = 'nf-Llama-3.1-70b-instruct'


options_zero_str = """
seed: 42
temperature: 0
"""
options_zero = yaml.safe_load(options_zero_str)

temperature_0 : int = 0
SEED: int = 42
SEED2: int = 43
MAX10: int = 10
TOPP1: int = 1

options_creative_str = """
seed: 42
temperature: 0.7
topp: 0.8
"""
options_creative = yaml.safe_load(options_creative_str)

options_large_str = """
seed: 42
temperature: 0
num_predict: 2000
"""
options_large = yaml.safe_load(options_large_str)

#load environment variables:
api_key = os.environ.get('sjoerd_key')

#setttings:
api_endpoint = "https://ai-research-proxy.azurewebsites.net/chat/completions"
api_endpoint_embed = "https://ai-research-proxy.azurewebsites.net/embeddings"
####### API REQUEST FORMATTING ######
headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer " + api_key
}

In [46]:
#classify Boukes with Llama3.3:70b

pubspherepromptsrun1 = ['rationality_simple2']
#pubspherepromptsrunall = ['rationality_simple2', 'rationality_jaidka', 'incivility_simple2', 'incivility_jaidka',  'civility_jaidka', 'interactivity_acknowledgement_simple2', 'interactivity_acknowledgement_jaidka', 'political_ideology_US', 'political_post', 'political_post_jaidka' ]  

predictions2: typing.Dict[str, np.ndarray] = {
    label: (
        src.PromptClassify
        .from_json(path)
        (boukesT["commentText"], model=MODEL33large, options=options_low)
    )
    for label, path in CFG.prompt_classify_files.items() if label in pubspherepromptsrun1
}

classifying rationality_simple2:   0%|          | 2/3862 [01:17<41:34:44, 38.78s/it]


KeyboardInterrupt: 

In [ ]:
#use GPT4o to compile paraphrased prompts::
def load_json(path: str):
    with open(path, encoding='utf-8') as fp:
        return json.load(fp)
    

#pubspherepromptsrunall = ['rationality_simple2', 'rationality_jaidka', 'incivility_simple2', 'incivility_jaidka',  'civility_jaidka', 'interactivity_simple2', 'interactivity_acknowledgement_jaidka', 'political_ideology_US', 'political_post', 'political_post_jaidka' ]  

pubsphereparaphraserun1 = ['rationality_simple2', 'incivility_simple2', 'interactivity_acknowledgement_simple2', 'political_ideology_US', 'political_post'] 



#chunked_result: typing.List[pd.DataFrame] = []

for label, path in CFG.prompt_classify_files.items():
    if label in pubsphereparaphraserun1: 
        template = load_json(path).get('template')
        classes = load_json(path).get('classes')
        retry_count = 0
        max_retries = 5
        while retry_count < max_retries:
            try: 
                response = requests.post(
                        url=api_endpoint,
                        headers=headers,
                        json={
                            'model': MODELgpt4o,
                            'messages': [
                                {
                                    "role": "system",
                                    "content": 'You restate a prompt in different words while preserving its meaning and formatting. The prompt is: "{}"'
                                },
                                {
                                    "role": "user",
                                    "content": 'You restate a prompt in different words while preserving its meaning and formatting. The prompt is: "{}"'.format(template)
                                }
                            ],
                            'temperature': 0.7,
                            'top_p': 0.8,  
                            'seed': SEED
                        }
                    )  
                if response.status_code == 200:
                    data_response = response.json()
                    chunked_result.append(
                        pd.DataFrame(
                            data=[[label, SEED, data_response["choices"][0]["message"]["content"]]],                                
                            columns=['prompt_label', 'seed', 'paraphrased_prompt']
                        )
                    )
                    break  # Exit retry loop for this label
                elif response.status_code == 429:
                    retry_count += 1
                    wait_time = 20
                    print(f"Rate limit exceeded. Retrying in {wait_time} seconds...")
                    time.sleep(wait_time)
                elif response.status_code == 500:
                    retry_count += 1
                    wait_time = 20
                    print(f"Failed to connect to API. Status code: {response.status_code}. Retrying in {wait_time} seconds...")
                    time.sleep(wait_time)
                else:
                    print(f"Failed to connect to API. Status code: {response.status_code}")
                    print(response.text)
                    # Optionally append a row with error info
                    chunked_result.append(
                        pd.DataFrame(
                            data=[[label, f"ERROR: {response.status_code}"]],
                            columns=['prompt_label', 'paraphrased_prompt']
                        )
                    )
                    break
            except requests.exceptions.RequestException as e:   
                print(f"Failed to connect to API: {e}")
                retry_count += 1
                wait_time = 60
                print(f"Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
        else:
            # If all retries failed, append a row with error info
            chunked_result.append(
                pd.DataFrame(
                    data=[[label, "ERROR: Max retries exceeded"]],
                    columns=['prompt_label', 'paraphrased_prompt']
                )
            )

para1 = pd.concat(chunked_result, ignore_index=True)
print(para1)
# ...existing code...

                            prompt_label  seed  \
0                     incivility_simple2    43   
1  interactivity_acknowledgement_simple2    43   
2                  political_ideology_US    43   
3                         political_post    43   
4                    rationality_simple2    43   
5                     incivility_simple2    42   
6  interactivity_acknowledgement_simple2    42   
7                  political_ideology_US    42   
8                         political_post    42   
9                    rationality_simple2    42   

                                                                                    paraphrased_prompt  
0  Is this comment uncivil?\nGuidelines: Mark Yes (1) if the comment contains name-calling, insults...  
1  Is the comment a reference to, an acknowledgment of, or a reply to another user's remark?\nInstr...  
2  Categorize the given message according to its ideological stance as liberal (0), neutral (1), or...  
3  Determine whether the give

In [43]:
para1

,prompt_label,seed,paraphrased_prompt
0,incivility_simple2,43,"Is this comment uncivil?\nGuidelines: Mark Yes (1) if the comment contains name-calling, insults..."
1,interactivity_acknowledgement_simple2,43,"Is the comment a reference to, an acknowledgment of, or a reply to another user's remark?\nInstr..."
2,political_ideology_US,43,"Categorize the given message according to its ideological stance as liberal (0), neutral (1), or..."
3,political_post,43,Determine whether the given message should be categorized as political (1) or non-political (0)....
4,rationality_simple2,43,Assess whether the comment offers logical evaluation.\nGuidelines: Assign Yes (1) if the comment...
5,incivility_simple2,42,Determine if the comment shows incivility.\nGuidelines: Assign Yes (1) if the comment contains n...
6,interactivity_acknowledgement_simple2,42,"Determine if this comment is referencing, recognizing, or replying to another user's comment.\nG..."
7,political_ideology_US,42,"Determine whether the given text aligns with liberal views (0), is ideologically neutral (1), or..."
8,political_post,42,Determine whether the given message is political (1) or non-political (0). A message is consider...
9,rationality_simple2,42,Evaluate whether the comment offers logical examination.\nInstructions: Assign Yes (1) if the co...


This rewording appears as dissimilar as the Jaidka:
-Boukes: "Does this comment provide rational analysis?\nInstructions: Code Yes (1) if the comment includes:\nContext or background,\nEvidence (facts, sources, authorities),\nReasoning or structured argument.\nCode No (0) if these are absent\n\\n\\nRespond with only the predicted class (0 or 1) of the request.\\n\\nText: {text}\\nClass:",
   
-Jaidka: "This tweet is a reply on Twitter (i.e., a Tweet) to a United States member of the Congress. Please classify this tweet according to whether it has a justification. Read the tweet. Determine which category best describes the tweet. Code YES (1): If this tweet contains personal feelings or experiences. Also code YES (1) If this tweet contains facts, links or evidence from other sources. Code NO (0) If this tweet does not offer a justification. \n\\n\\nRespond with only the predicted class (0 or 1) of the request.\\n\\nText: {text}\\nClass:",

-GPT4o_temp0.7_topp0.8_seed1: 'Evaluate whether the comment offers logical examination. \n\nGuidelines: Assign Yes (1) if the comment contains:\n- Context or background information,\n- Supporting evidence (facts, references, experts),\n- Logical reasoning or organized argumentation.\n\nAssign No (0) if these elements are missing.\n\nReply with just the anticipated category (0 or 1) for the request.\n\nText: {text}\nClass:',

-GPT4o_temp0.7_topp0.8_seed2:  'Assess whether the comment offers logical evaluation.\nGuidelines: Assign Yes (1) if the comment contains:\nContext or background information,\nSupporting evidence (facts, references, expert opinions),\nLogical reasoning or organized argumentation.\nAssign No (0) if these elements are missing.\n\\n\\nProvide solely the anticipated classification (0 or 1) of the inquiry.\\n\\nText: {text}\\nClass:',
   

-GPT4o_temp0.5_topp0.5: 'Evaluate whether the comment offers logical examination.\nInstructions: Assign Yes (1) if the comment contains:\nContext or background information,\nEvidence (facts, references, experts),\nLogical reasoning or organized argument.\nAssign No (0) if these elements are missing.\n\\n\\nReply with just the anticipated category (0 or 1) of the inquiry.\\n\\nText: {text}\\nClass:',
    

-> maybe we can generate two paraphrases per boukes prompt with different seeds, which look like they would be rather similar to each other to get a grasp of very small prompt differences, compared to different codebooks or paraphrases of a prompt.
also helps to address the potential critizism that our prompts favor Llama, since most prompt engineering was done on lama (but prompt wording of both simple and paraphrased prompts are based on GPT4o)

    
      

In [45]:
#save the paraphrased prompts to parquet file:
para1.to_parquet(f'{CFG.report_dir}/Boukes_paraphrased_prompts.parquet', index=False)
#save to json file:
para1.to_json(f'{CFG.report_dir}/Boukes_paraphrased_prompts.json', orient='records', force_ascii=False, indent=4)